# ML API Specification for EO Data Cubes
### Brian Pondi and Rolf Simoes

## Setup and Connection

First, we load the required libraries and connect to the OpenEO backend.
Install the `openeo` package variant that's on this GitHub repository: https://github.com/PondiB/openeo-python-client

In [2]:
import openeo # type: ignore

In [3]:
connection = openeo.connect(url="http://127.0.0.1:8000")

In [4]:
connection.authenticate_basic("brian", "123456")

<Connection to 'http://127.0.0.1:8000/' with BasicBearerAuth>

In [5]:
connection.list_collection_ids()

['mpc-landsat-c2-l2',
 'mpc-sentinel-2-l2a',
 'mpc-sentinel-1-grd',
 'mpc-sentinel-1-rtc']

In [6]:
connection.describe_collection("mpc-sentinel-2-l2a")

{'type': 'Collection',
 'stac_version': '1.0.0',
 'id': 'mpc-sentinel-2-l2a',
 'title': 'Sentinel-2 Level-2A',
 'description': 'The [Sentinel-2](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) program provides global imagery in thirteen spectral bands at 10m-60m resolution and a revisit time of approximately five days.  This dataset represents the global Sentinel-2 archive, from 2016 to the present, processed to L2A (bottom-of-atmosphere) using [Sen2Cor](https://step.esa.int/main/snap-supported-plugins/sen2cor/) and converted to [cloud-optimized GeoTIFF](https://www.cogeo.org/) format.',
 'license': 'proprietary',
 'providers': [{'url': 'https://sentinel.esa.int/web/sentinel/missions/sentinel-2',
   'name': 'ESA',
   'roles': ['producer', 'licensor']},
  {'url': 'https://www.esri.com/', 'name': 'Esri', 'roles': ['processor']},
  {'url': 'https://planetarycomputer.microsoft.com',
   'name': 'Microsoft',
   'roles': ['host', 'processor']}],
 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
  'temporal': {'interval': [['2015-06-27T10:25:31Z', {}]]}},
 'links': [{'rel': 'root',
   'href': 'http://127.0.0.1:8000/',
   'type': 'application/json'},
  {'rel': 'self',
   'href': 'http://127.0.0.1:8000/collections/mpc-sentinel-2-l2a',
   'type': 'application/json'},
  {'rel': 'parent',
   'href': 'http://127.0.0.1:8000/',
   'title': 'Parent',
   'type': 'application/geo+json'},
  {'rel': 'items',
   'href': 'http://127.0.0.1:8000/collections/mpc-sentinel-2-l2a/items',
   'title': 'Items',
   'type': 'application/geo+json'}],
 'cube:dimensions': {'x': {'type': 'spatial',
   'axis': 'x',
   'extent': [-180, 180],
   'reference_system': 42001},
  'y': {'type': 'spatial',
   'axis': 'y',
   'extent': [-90, 90],
   'reference_system': 42001},
  't': {'type': 'temporal',
   'axis': 't',
   'extent': ['2015-06-27T10:25:31Z', {}]},
  'bands': {'type': 'bands',
   'values': ['b01',
    'b02',
    'b03',
    'b04',
    'b05',
    'b06',
    'b07',
    'b08',
    'b8a',
    'b09',
    'b11',
    'b12',
    'cloud']}}}

## Explore Available Processes

Let's examine the available processes on the backend, particularly focusing on ML-related ones.


In [7]:
process_ids = [process["id"] for process in connection.list_processes()]
print("Available processes on this backend:")
for process_id in process_ids:
    print(f"- {process_id}")

Available processes on this backend:
- load_collection
- mlm_class_random_forest
- mlm_class_svm
- mlm_class_xgboost
- ml_class_mlp
- mlm_class_tempcnn
- mlm_class_tae
- mlm_class_lighttae
- ml_fit
- ml_predict
- ml_predict_probabilities
- ml_uncertainty_class
- ml_smooth_class
- ml_label_class
- cube_regularize
- ndvi
- save_result
- load_result
- export_cube
- import_cube
- export_ml_model
- import_ml_model
- save_ml_model
- load_ml_model


## Examine ML Process Details

Let's look at the details of some ML processes to understand their parameters and requirements.


In [8]:
connection.describe_process("mlm_class_random_forest")

{'id': 'mlm_class_random_forest',
 'summary': 'Initialize a Random Forest classification model',
 'description': 'Creates and configures a Random Forest classification model. Parameters such as the number of trees and maximum depth can be specified.',
 'categories': ['machine learning'],
 'experimental': True,
 'parameters': [{'name': 'num_trees',
   'description': 'The number of trees in the Random Forest.',
   'schema': {'type': 'integer', 'minimum': 1, 'default': 100}},
  {'name': 'max_variables',
   'description': 'Specifies how many split variables will be used at a node.\n\nThe following options are available:\n\n- *integer*: The given number of variables are considered for each split.\n- `log2`: The logarithm with base 2 of the number of variables are considered for each split.\n- `sqrt`: The square root of the number of variables are considered for each split. This is often the default for classification.',
   'optional': True,
   'default': 'sqrt',
   'schema': [{'type': 'integer', 'minimum': 1},
    {'type': 'string', 'enum': ['log2', 'sqrt']}]},
  {'name': 'seed',
   'description': 'A randomization seed to use for the random sampling in training. If not given or `null`, no seed is used and results may differ on subsequent use.',
   'optional': True,
   'default': {},
   'schema': {'type': ['integer', 'null']}}],
 'returns': {'description': 'An untrained Random Forest classification model instance.',
  'schema': {'type': 'object', 'subtype': 'ml-model'}},
 'links': [{'href': 'https://doi.org/10.1023/A:1010933404324',
   'title': 'Breiman (2001): Random Forests',
   'type': 'text/html',
   'rel': 'about'}]}

In [9]:
connection.describe_process("mlm_class_tempcnn")

{'id': 'mlm_class_tempcnn',
 'summary': 'Initialize a Temporal Convolutional Neural Network (TempCNN) classification model',
 'description': 'Creates and configures a Temporal Convolutional Neural Network (TempCNN) model for time-series data. Parameters such as the convolutional architecture, dense layers, optimizer, and learning rate schedule can be specified.',
 'categories': ['machine learning'],
 'experimental': True,
 'parameters': [{'name': 'cnn_layers',
   'description': 'A list specifying the number of filters in each convolutional layer. Defaults to [64, 64, 64].',
   'optional': True,
   'default': [64, 64, 64],
   'schema': {'type': 'array',
    'items': {'type': 'integer', 'minimum': 1},
    'minItems': 1}},
  {'name': 'cnn_kernels',
   'description': 'A list specifying the kernel size for each convolutional layer. Must match the number of CNN layers. Defaults to [5, 5, 5].',
   'optional': True,
   'default': [5, 5, 5],
   'schema': {'type': 'array',
    'items': {'type': 'integer', 'minimum': 1},
    'minItems': 1}},
  {'name': 'cnn_dropout_rates',
   'description': 'A list of dropout rates for each convolutional layer. Must match the number of CNN layers. Defaults to [0.2, 0.2, 0.2].',
   'optional': True,
   'default': [0.2, 0.2, 0.2],
   'schema': {'type': 'array',
    'items': {'type': 'number', 'minimum': 0, 'maximum': 1},
    'minItems': 1}},
  {'name': 'dense_layer_nodes',
   'description': 'The number of nodes in the dense layer following the convolutional layers. Defaults to 256.',
   'optional': True,
   'default': 256,
   'schema': {'type': 'integer', 'minimum': 1}},
  {'name': 'dense_layer_dropout_rate',
   'description': 'The dropout rate for the dense layer. Defaults to 0.5.',
   'optional': True,
   'default': 0.5,
   'schema': {'type': 'number', 'minimum': 0, 'maximum': 1}},
  {'name': 'optimizer',
   'description': "The optimizer to use for training. Defaults to 'adam'. Supported values include 'adam', 'adabound', 'adabelief', 'madagrad', 'nadam', 'qhadam', 'radam', 'swats', and 'yogi'.",
   'optional': True,
   'default': 'adam',
   'schema': {'type': 'string',
    'enum': ['adam',
     'adabound',
     'adabelief',
     'madagrad',
     'nadam',
     'qhadam',
     'radam',
     'swats',
     'yogi']}},
  {'name': 'learning_rate',
   'description': 'The initial learning rate for training. Defaults to 0.0005.',
   'optional': True,
   'default': 0.0005,
   'schema': {'type': 'number', 'minimum': 0}},
  {'name': 'epsilon',
   'description': 'The epsilon value for numerical stability in optimizers. Defaults to 1e-8.',
   'optional': True,
   'default': 1e-08,
   'schema': {'type': 'number', 'minimum': 0}},
  {'name': 'weight_decay',
   'description': 'The weight decay (L2 penalty) value for regularization. Defaults to 1e-6.',
   'optional': True,
   'default': 1e-06,
   'schema': {'type': 'number', 'minimum': 0}},
  {'name': 'lr_decay_epochs',
   'description': 'The number of epochs after which the learning rate is decayed. Defaults to 1.',
   'optional': True,
   'default': 1,
   'schema': {'type': 'integer', 'minimum': 1}},
  {'name': 'lr_decay_rate',
   'description': 'The rate at which the learning rate is decayed after the specified number of epochs. Defaults to 0.95.',
   'optional': True,
   'default': 0.95,
   'schema': {'type': 'number', 'minimum': 0, 'maximum': 1}},
  {'name': 'epochs',
   'description': 'The number of training epochs. Defaults to 150.',
   'optional': True,
   'default': 150,
   'schema': {'type': 'integer', 'minimum': 1}},
  {'name': 'batch_size',
   'description': 'The size of batches for training. Defaults to 64.',
   'optional': True,
   'default': 64,
   'schema': {'type': 'integer', 'minimum': 1}},
  {'name': 'seed',
   'description': 'Seed for the random number generator. If `null`, no seed is used, and results may vary.',
   'optional': True,
   'default': {},
   'schema': {'type': ['integer', 'null']}}],
 'returns': {'description': 'An untrained TempCNN classif

## Load and Prepare Data

We'll load Sentinel-2 data and prepare it for our analysis.

In [10]:
datacube = connection.load_collection(
    collection_id="mpc-sentinel-2-l2a",
    spatial_extent={"west": -63.6078, "south": -8.95630, "east": -63.25790, "north": -8.72290},
    temporal_extent=["2022-01-01", "2022-12-31"]
)

In [11]:
datacube = datacube.process(
    process_id="cube_regularize",
    arguments={
        "data": datacube,
        "period": "P1M",  # Monthly regularization
        "resolution": 30
    }
)

## Load Training Data

We'll load the pre-processed training data for deforestation in Rondonia.


In [12]:
serialized_data = connection.readRDS("./data/monthly_rondonia_data.rds")

## Initialize and Train the Model

Now we'll initialize the Temporal CNN model and train it with our data.

In [13]:
tempcnn_model_init = connection.mlm_class_tempcnn(
    optimizer="adam",
    learning_rate=0.0005,
    seed=42
)

Fit the model using the training dataset

In [14]:
tempcnn_model = tempcnn_model_init.fit(
    training_set=serialized_data,
    target="label"
)

## Make Predictions

Apply the trained model to make predictions on new data.


In [15]:
datacube =  tempcnn_model.predict(datacube)

## Save the Model

Save the trained model for future use.

In [16]:
tempcnn_model.save_ml_model(name ="tempcnn_rondonia", tasks=["classification"]
                            #, options={"mlm:accelerator":"macos-arm", "mlm:framework":"Torch"}
                            )

## Save and Execute Results

Finally, we'll save the prediction results and execute the job.


In [17]:
result = datacube.save_result(
    format="GeoTiff"
)

In [18]:
result

In [19]:
job = result.create_job(
    title="Deforestation Prediction in Rondonia",
    description="Using TempCNN model to predict deforestation in Rondonia"
)

In [ ]:
job.start_and_wait()
results = job.get_results()

0:00:00 Job '170690b6707deff13a9d97b6ddd22baf': send 'start'
0:00:12 Job '170690b6707deff13a9d97b6ddd22baf': running (progress N/A)
0:00:17 Job '170690b6707deff13a9d97b6ddd22baf': running (progress N/A)
0:00:24 Job '170690b6707deff13a9d97b6ddd22baf': running (progress N/A)
0:00:31 Job '170690b6707deff13a9d97b6ddd22baf': running (progress N/A)
0:00:41 Job '170690b6707deff13a9d97b6ddd22baf': running (progress N/A)
0:00:53 Job '170690b6707deff13a9d97b6ddd22baf': running (progress N/A)
0:01:09 Job '170690b6707deff13a9d97b6ddd22baf': running (progress N/A)
0:01:28 Job '170690b6707deff13a9d97b6ddd22baf': running (progress N/A)
0:01:52 Job '170690b6707deff13a9d97b6ddd22baf': running (progress N/A)
0:02:22 Job '170690b6707deff13a9d97b6ddd22baf': running (progress N/A)
0:02:59 Job '170690b6707deff13a9d97b6ddd22baf': running (progress N/A)
0:03:46 Job '170690b6707deff13a9d97b6ddd22baf': running (progress N/A)
0:04:44 Job '170690b6707deff13a9d97b6ddd22baf': running (progress N/A)
0:05:44 Job '170

In [ ]:
results.download_files("data/output")

## Conclusion

This example demonstrated how to:
1. Connect to an OpenEO backend
2. Load and prepare training data
3. Define a Temporal CNN model architecture
4. Train the model
5. Make predictions on new data
6. Save the results

The trained model can now be used for making predictions on new time series data. 